In [1]:
## read the file (.pairs)

In [2]:
import numpy.random as random
import numpy as np
import math
import networkx as nx
from scipy import sparse as sp
import random

In [30]:
ntwk = nx.read_gml('sourcefile/karate.gml', 'id')
n_A = nx.adjacency_matrix(ntwk)

In [31]:
print(n_A.toarray())

[[0 1 1 ... 1 0 0]
 [1 0 1 ... 0 0 0]
 [1 1 0 ... 0 1 0]
 ...
 [1 0 0 ... 0 1 1]
 [0 0 1 ... 1 0 1]
 [0 0 0 ... 1 1 0]]


In [57]:
#n is number of elements, cn, constrainumber
def initial_grouping(n, cn):

    #get list of nodes
    nodes = [node for node in range(0,n)]
    
    #assign group number to each nodes
    group_assign = [random.randrange(0,cn) for g_num in range(len(nodes))]
    
    return nodes, group_assign

#partition is group-assigned list
def partition_matrix(partition) :
    n = len(partition)
    data= np.ones(n)
    #print(data)
    ij = np.array([partition, list(range(0,n))])
    #print(ij)
    grouping_matrix = sp.csr_matrix((data,ij))
    return grouping_matrix

# s = partition matrix, k = degree matrix , m = number of edge 

#generating degree matrix 
def degree_matrix(adj) :
    d_mat = n_A.sum(axis=1)
    return d_mat


def modularity_mat(adj, k, m ):
    
    null_model  = np.dot(k,k.transpose())/(2*m)
    m_m= (adj - null_model)
    
    return m_m

#calculate modularity
def get_modularity(adj, s, k, m):
    
    # formula modularity = (Adj * p_matrix - (d_matrix)^2 * p_matrix/2m) / 2m 
    #adj_partition  = (s* adj *s.transpose()).diagonal().sum()
    #deg_par = np.array(s*k)
    #degree_partition = np.square(deg_par).sum()/(2*m) 
    #modularity
    #Q = (adj_partition - degree_partition) /(2*m)
    
    mod_matrix = modularity_mat(adj, k, m )
    modul = (s*mod_matrix*s.transpose()).diagonal().sum()
    Q = modul/(2*m)
    
    return Q

def patching(group_list, cn) :
    
    n_nodes =len(group_list)
    #cn is constrained number of groups
    n_groups = cn
    #get index and value that want to change
    
    idx = random.choice([x for x in range(0, n_nodes)])
    val = group_list[idx]
    
    # choice here needs reworking - ??
    # calculate change in modularity if we do switch groups
    # if its bad, don't do it - ??
    n_val = random.choice([y for y in range(0, n_groups) if y != val])
    
    group_list[idx] = n_val
    patched_group = group_list
    
    return patched_group



def annealing_simulating(G, initial_temp, adj,n, cn,  s, k, m, cooling_constant, iteration) :
    G = G
    svg_group_list = []
    modul_list = []
    old_Q = None
    group_list = None
    for i in range(iteration) :
        
        temp = new_temp(cooling_constant, initial_temp, i)
        acceptance = False 
        num_rejection = -1
        #n numberof nodes, cn constrained number of groups
        rej_threshold = cal_rej_thres(n,cn)
        #print(rej_threshold)
        
        while acceptance is False :
            
            num_rejection +=1
            #print(old_Q)
            
            #when the very first
            if old_Q is None :
                node_list , group_list = initial_grouping(n,cn)
                s = partition_matrix(group_list)
                old_Q = get_modularity(adj, s, k, m)
                
            
            # patching new grouping list
            new_grouping = patching(group_list, cn)
            #print(new_grouping)
            
            # new grouping matrix
            new_s = partition_matrix(new_grouping)
            
            # new modularity with new grouping matrix
            comm_0 = [x+1 for x, y in enumerate(new_grouping) if y == 0]
            comm_1 = [x+1 for x, y in enumerate(new_grouping) if y == 1]
            new_Q = nx.community.modularity(G, [set(comm_0), set(comm_1)])

            #new_Q = get_modularity(adj,new_s,k,m)
            #print(new_Q)
            
            acceptance = check_acceptance(old_Q, new_Q, temp)
            
            #print(acceptance)
            #print(num_rejection)
            if num_rejection > rej_threshold:
                break
            
            
            if acceptance is True :
                
                #if new_grouping in svg_group_list :
                    #acceptance = False
                #else :
                svg_group_list.append(new_grouping)
                modul_list.append(new_Q)
                print(modul_list[-1])
                old_Q = new_Q
                # accepted grouping would be old grouping and update group based on this grouping 
                group_list = new_grouping
                
    opt = max(modul_list)
    idx_opt = modul_list.index(opt)
    
    opt_group = svg_group_list[idx_opt]
    
    opt_grouping = svg_format(opt_group) 
    opt_Q = opt
    
    return opt_grouping, opt_Q, svg_group_list, modul_list
            


# get updated temperature
def new_temp(cooling_constant, prev_temp, times) :
    new_temp = (math.pow(cooling_constant,times)) * prev_temp
    return new_temp 

# check acceptance between old groups and new groups
def check_acceptance(modul_old, modul_new  , temp) :
    if modul_new > modul_old :
        return True
    else :
        new_accept = math.exp((modul_new - modul_old) * temp)
        #modul_new- modul_old is negative value
        # when temp is higher , new_accept is getting smaller 
    return (np.random.random_sample() < new_accept)  #get true or false



#calculate rejection threshold 

def cal_rej_thres(n, nc):
    
    num_pathing = n*nc
    
    num_of_moves = num_pathing
    
    confi = 1 - math.log(0.05)/math.log(num_of_moves)
    
    return confi *num_of_moves * math.log(num_of_moves)

#saving format for check group easily 

def svg_format(group_list):
    fin_set ={}
    for idx, gnum in enumerate(group_list):
        if gnum in fin_set :
            fin_set[gnum].append(idx)
        else :
            fin_set[gnum] =[idx]
    return fin_set



## Simulated Annealing for modularity optimization (Operating code)

### I) Setup

In [58]:
G = nx.read_gml('sourcefile/karate.gml', 'id')
n_A = nx.adjacency_matrix(G).toarray()
n = G.number_of_nodes()
m = sum([x[1] for x in G.degree()])
k = degree_matrix(n_A)
cn = 2
s = partition_matrix(initial_grouping(n, cn)[1])

In [59]:
print(G.nodes())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]


In [62]:
opt_grouping, opt_Q, saving_group, modul_list = annealing_simulating(G, initial_temp = 40, adj = n_A,n =n, cn=2,  s =s, k =k,m= m, cooling_constant = 1.0015,  iteration =1000) 

0.06311637080867892
0.06632149901380711
0.047255095332018784
0.043063773833004866
0.06311637080867878
0.08407297830374791
0.07881328073635804
0.10971400394477361
0.07486850756081562
0.07684089414858683
0.14981919789612091
0.1291913214990141
0.06311637080867867
0.04240631163708102
0.0472550953320185
0.055884286653517565
0.09853714661407029
0.06377383300460264
0.06278763971071705
0.025558842866535272
-0.012820512820512681
-0.014135437212360184
-8.218277449041883e-05
-0.0007396449704141745
-0.04372123602892808
-0.04142011834319504
-0.025723208415515855
-0.0007396449704140151
-0.10264628533859288
-0.03558514135437184
-0.00665680473372768
-0.015779092702169473
-0.014135437212360175
-0.025723208415515907
-0.004026955950032765
-0.0066568047337276825
0.02161406969099284
-0.0020545693622616157
0.0018902038132807965
-0.018491124260354895
-0.023750821827744736
-0.010601577909270054
-0.017094017094016832
-0.028928336620644028
-0.04569362261669923
-0.061143984220906916
-0.03550295857988132
-0.02596

In [63]:
opt_grouping
#a = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1]
#{0:0,1,2,3,4,5,6,7,8,9,10,11,12,13,16,17,19,21
# 1:14,15,16,20,22,23,24,25,26,27,28,29,30,31,32,33}

{1: [0, 1, 2, 3, 4, 6, 10, 11, 15, 17, 19, 22, 24, 26, 28, 29, 30],
 0: [5, 7, 8, 9, 12, 13, 14, 16, 18, 20, 21, 23, 25, 27, 31, 32, 33]}

## Simulated Annealing

In [16]:
def annealing_simulating(initial_temp, adj,n, cn,  s, k, m, cooling_constant, iteration) :
    
    svg_group_list = []
    modul_list = []
    old_Q = None
    group_list = None
    for i in range(iteration) :
        
        temp = new_temp(cooling_constant, initial_temp, i)
        acceptance = False 
        num_rejection = -1
        #n numberof nodes, cn constrained number of groups
        rej_threshold = cal_rej_thres(n,cn)
        #print(rej_threshold)
        
        while acceptance is False :
            
            num_rejection +=1
            #print(old_Q)
            
            #when the very first
            if old_Q is None :
                node_list , group_list = initial_grouping(n,cn)
                s = partition_matrix(group_list)
                old_Q = get_modularity(adj, s, k, m)
                
            
            # patching new grouping list
            new_grouping = patching(group_list, cn)
            #print(new_grouping)
            
            # new grouping matrix
            new_s = partition_matrix(new_grouping)
            
            # new modularity with new grouping matrix
            new_Q = get_modularity(adj,new_s,k,m)
            #print(new_Q)
            
            acceptance = check_acceptance(old_Q, new_Q, temp)
            
            #print(acceptance)
            #print(num_rejection)
            if num_rejection > rej_threshold:
                break
            
            
            if acceptance is True :
                
                #if new_grouping in svg_group_list :
                    #acceptance = False
                #else :
                svg_group_list.append(new_grouping)
                modul_list.append(new_Q)
                #print(modul_list[-1])
                old_Q = new_Q
                group_list = new_grouping
                
    opt = max(modul_list)
    idx_opt = modul_list.index(opt)
    
    opt_group = svg_group_list[idx_opt]
    
    opt_grouping = svg_format(opt_group) 
    opt_Q = opt
    
    return opt_grouping, opt_Q, svg_group_list, modul_list
            
        
        

## Getting modularity 

In [17]:
# s = partition matrix, k = degree matrix , m = number of edge 
m = (sumit/2)

def get_modularity(adj, s, k, m ):
    
    # formula modularity = (Adj * p_matrix - (d_matrix)^2 * p_matrix/2m) / 2m 
    
    adj_partition  = (s* adj *s.transpose()).diagonal().sum()
    deg_par = np.array(s*k)
    degree_partition = np.square(deg_par).sum()/(2*m) 
    
    #modularity
    Q = (adj_partition - degree_partition) /(2*m)
    
    return Q
    

NameError: name 'sumit' is not defined

## Degree matrix

In [18]:
def degree_matrix(adj) :
    d_mat = n_A.sum(axis=1)
    return d_mat

## Initial grouping

In [19]:
#n is number of elements, cn, constrainumber
def initial_grouping(n,cn):
    
    
    #get list of nodes
    nodes = [node for node in range(0,n)]
    
    #assign group number to each nodes
    group_assign = [random.randrange(0,cn) for g_num in range(len(nodes))]
    
    return nodes, group_assign
    

## Getting S(Partition matrix) 

In [20]:
def partition_matrix(partition) :
    print(partition)
    n = len(partition)
    data= np.ones(n)
    print(data)
    ij = np.array([partition, list(range(0,n))])
    print(ij)
    grouping_matrix = sp.csr_matrix((data,ij))
    return grouping_matrix

## Saving format for checking group easily

In [21]:
def svg_format(group_list):
    fin_set ={}
    for idx, gnum in enumerate(group_list):
        if gnum in fin_set :
            fin_set[gnum].append(idx)
        else :
            fin_set[gnum] =[idx]
    return fin_set

## Pathcing the group move singe node into other groups

In [22]:
def patching(group_list, cn) :
    
    n_nodes =len(group_list)
    #cn is constrained number of groups
    n_groups = cn
    #get index and value that want to change
    
    idx = random.choice([x for x in range(0, n_nodes)])
    val = group_list[idx]
    
    n_val = random.choice([y for y in range(0, n_groups) if y != val])
    
    group_list[idx] = n_val
    patched_group = group_list
    
    return patched_group
    